In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
from torch.nn.functional import softmax
import re

# Cargar el tokenizer y el modelo preentrenado de BERT para clasificación de sentimientos
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Función para limpiar el texto eliminando caracteres no útiles
def limpiar_texto(texto):
    # Eliminar números, símbolos y caracteres no alfabéticos
    texto_limpio = re.sub(r'[^a-zA-Z\s]', '', texto)
    # Convertir el texto a minúsculas
    texto_limpio = texto_limpio.lower()
    return texto_limpio

# Función para preprocesar el texto y tokenizarlo
def preprocesar(texto):
    texto_limpio = limpiar_texto(texto)
    inputs = tokenizer(texto_limpio, return_tensors='pt', truncation=True, padding=True, max_length=512)
    return inputs

# Función para realizar la predicción con BERT
def predecir_sentimiento(comentario):
    # Preprocesar el comentario
    inputs = preprocesar(comentario)
    # Realizar la predicción
    outputs = model(**inputs)
    logits = outputs.logits
    # Aplicar softmax para obtener probabilidades
    probabilidades = softmax(logits, dim=1)
    # Predecir la clase con mayor probabilidad
    prediccion = torch.argmax(probabilidades, dim=1)
    return prediccion.item(), probabilidades

# Función para interpretar la predicción
def interpretar_prediccion(prediccion):
    sentimientos = {
        0: "Muy negativo",
        1: "Negativo",
        2: "Neutral",
        3: "Positivo",
        4: "Muy positivo"
    }
    return sentimientos.get(prediccion, "Indeterminado")

# Comentarios de ejemplo
comentarios = [
    "El producto llegó a tiempo y en excelentes condiciones, lo recomiendo ampliamente.",
    "No tiene buen sabor, esperaba mucho más por el precio que pagué.",
    "Esta AGRIO ese LICOR",
    "AD113131313.ESTUVO MUY BUENO !#1!#!"
]

# Predicción de sentimientos para cada comentario
for comentario in comentarios:
    prediccion, probabilidades = predecir_sentimiento(comentario)
    sentimiento = interpretar_prediccion(prediccion)
    print(f"Comentario: {comentario}")
    print(f"Predicción: {sentimiento}")
    print(f"Probabilidades: {probabilidades}")
    print('-' * 50)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

Comentario: El producto llegó a tiempo y en excelentes condiciones, lo recomiendo ampliamente.
Predicción: Muy positivo
Probabilidades: tensor([[0.0029, 0.0034, 0.0423, 0.3270, 0.6244]], grad_fn=<SoftmaxBackward0>)
--------------------------------------------------
Comentario: No tiene buen sabor, esperaba mucho más por el precio que pagué.
Predicción: Negativo
Probabilidades: tensor([[0.2791, 0.5485, 0.1636, 0.0075, 0.0013]], grad_fn=<SoftmaxBackward0>)
--------------------------------------------------
Comentario: Esta AGRIO ese LICOR
Predicción: Muy negativo
Probabilidades: tensor([[0.2465, 0.1809, 0.1822, 0.1648, 0.2256]], grad_fn=<SoftmaxBackward0>)
--------------------------------------------------
Comentario: AD113131313.ESTUVO MUY BUENO !#1!#!
Predicción: Muy positivo
Probabilidades: tensor([[0.0095, 0.0077, 0.0432, 0.2450, 0.6945]], grad_fn=<SoftmaxBackward0>)
--------------------------------------------------


# USANDO BERT -> PARA ANALISIS DE COMENTARIOS

In [ ]:
# Instalar las librerías necesarias
!pip install transformers

# Importar librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch


In [ ]:
df = pd.read_excel("Comentarios_Licores_Rebalanceado.xlsx")  # Cambia el nombre a tu archivo
df['label'] = df['Etiqueta'].map({'Positivo': 0, 'Negativo': 1, 'Neutro': 2, 'Invalido': 3})  # Mapear etiquetas a números


In [ ]:
# Dividir los datos en entrenamiento y validación
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Comentario'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42
)


In [ ]:
# Tokenizar los textos usando BERT
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [ ]:
tokenizer

BertTokenizer(name_or_path='dccuchile/bert-base-spanish-wwm-cased', vocab_size=31002, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
# Crear un Dataset personalizado para PyTorch
class CommentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = CommentDataset(train_encodings, train_labels)
val_dataset = CommentDataset(val_encodings, val_labels)

In [ ]:
# Cargar el modelo BERT preentrenado
model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=4) # dccuchile/bert-base-spanish-wwm-cased' |bert-base-uncased


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    run_name="Mi_Entrenamiento_BERT",  # Nombre del experimento
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Configurar Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
# Entrenar el modelo
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.018900,0.005242
2,0.001300,0.007528
3,0.000600,0.000382


TrainOutput(global_step=645, training_loss=0.16333497316150586, metrics={'train_runtime': 142.2335, 'train_samples_per_second': 72.304, 'train_steps_per_second': 4.535, 'total_flos': 269531282827872.0, 'train_loss': 0.16333497316150586, 'epoch': 3.0})

In [ ]:
# Evaluar el modelo
predictions = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1)
print(classification_report(val_labels, preds.numpy(), target_names=['Positivo', 'Negativo', 'Neutro', 'Invalido']))


              precision    recall  f1-score   support

    Positivo       1.00      1.00      1.00       151
    Negativo       1.00      1.00      1.00       158
      Neutro       1.00      1.00      1.00       161
    Invalido       1.00      1.00      1.00       387

    accuracy                           1.00       857
   macro avg       1.00      1.00      1.00       857
weighted avg       1.00      1.00      1.00       857



In [ ]:
# Guardar el modelo
model.save_pretrained("bert_clasificador")
tokenizer.save_pretrained("bert_clasificador")

('bert_clasificador/tokenizer_config.json',
 'bert_clasificador/special_tokens_map.json',
 'bert_clasificador/vocab.txt',
 'bert_clasificador/added_tokens.json')

In [ ]:
# Guardar el modelo y el tokenizer
output_dir = "bert_clasificador_espanol"  # Nombre del directorio donde se guardará el modelo

# Guardar el modelo entrenado
model.save_pretrained(output_dir)

# Guardar el tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Modelo y tokenizer guardados en: {output_dir}")

Modelo y tokenizer guardados en: bert_clasificador_espanol


In [ ]:
from transformers import pipeline

# Cargar modelo y tokenizer
model = BertForSequenceClassification.from_pretrained("bert_clasificador")
tokenizer = BertTokenizer.from_pretrained("bert_clasificador")

# Crear pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

# Clasificar un nuevo comentario
comentario = "El licor estuvo agrio "
resultado = classifier(comentario)
print(resultado)

Device set to use cuda:0


[[{'label': 'LABEL_0', 'score': 0.00012817163951694965}, {'label': 'LABEL_1', 'score': 0.999541163444519}, {'label': 'LABEL_2', 'score': 0.00017278574523516}, {'label': 'LABEL_3', 'score': 0.00015786985750310123}]]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# Cargar modelo y tokenizer entrenados
model = BertForSequenceClassification.from_pretrained("bert_clasificador")
tokenizer = BertTokenizer.from_pretrained("bert_clasificador")

# Crear pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

# Clasificar un nuevo comentario
comentario = "ghsahfgsahjfgsajhf hola estuvo rico"
resultado = classifier(comentario)

# Mostrar resultados procesados
etiquetas = ['Positivo', 'Negativo', 'Neutro', 'Invalido']
for clase, puntaje in zip(etiquetas, resultado[0]):
    print(f"Clase: {clase}, Puntaje: {puntaje['score']:.4f}")

# Identificar la clase con mayor puntaje
clase_predicha = etiquetas[max(range(len(resultado[0])), key=lambda i: resultado[0][i]['score'])]
print(f"\nEl comentario pertenece a la clase: {clase_predicha}")

Device set to use cuda:0


Clase: Positivo, Puntaje: 0.9975
Clase: Negativo, Puntaje: 0.0006
Clase: Neutro, Puntaje: 0.0008
Clase: Invalido, Puntaje: 0.0010

El comentario pertenece a la clase: Positivo
